In [188]:
import os 
import pandas as pd
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import re

pd.set_option('display.max_columns',6)
pd.set_option('display.max_rows', 10)
path="C:\\Users\\yesta\\Desktop"
os.chdir(path)

In [189]:
#df = pd.read_excel("budget copy.xlsx", "Sheet2")

df = pd.read_excel("budget14oct.xlsx", "budget")

date_pattern = r'([0-9]{4}-[0-9]{2}-[0-9]{2})'
row1 = df.iloc[:,0]
#print(row1)

In [190]:
categories = df.iloc[1:14, 22:24]
categories.columns = ["expense", "amount"]
categories["amount"] = 0
categories = categories.sort_values(by=["expense"], ascending = True)

#a = pd.DataFrame({"expense": ["Rent", "Health"], "amount": [1, 2]})
print(categories)
#print(categories, a)
#result = categories.set_index('expense').add(a.set_index('expense'), fill_value=0).reset_index()
#print(result)

               expense  amount
7           Eating out       0
11       Entertainment       0
10  Gifts/celebrations       0
6            Groceries       0
4               Health       0
..                 ...     ...
1                 Rent       0
9             Shopping       0
5        Subscriptions       0
3               Travel       0
2            Utilities       0

[13 rows x 2 columns]


In [191]:
all_dates = {}

first_date = str(df.columns.values[0])[:10]
# print('asfaef', first_date)
all_dates[0] = first_date

i = 0
j = 1
for i in range(0, len(row1)):
    date = re.search(date_pattern, str(row1[i]))
   # print(date)
    if date:
       # print(date.group())
        all_dates[j] = date.group()
        j += 1

#for thing in all_dates:
#    print(thing, all_dates[thing])

#print(len(row1))

In [192]:
cycle_dates = {int(k/2):v for (k,v) in all_dates.items() if k%2==0}
print(cycle_dates)

offset = 3
total_cols = (len(df.columns)+1)
df.drop(df.iloc[:, 1:total_cols:offset], axis=1, inplace=True)

cycle = 30
no_cycles = int(df.index[-1] / cycle)
print(no_cycles)
print(df.index[-1])

{0: '2023-04-05', 1: '2023-04-19', 2: '2023-05-03', 3: '2023-05-17', 4: '2023-05-31', 5: '2023-06-14', 6: '2023-06-28', 7: '2023-07-12', 8: '2023-07-26', 9: '2023-08-09', 10: '2023-08-23', 11: '2023-09-06', 12: '2023-09-20', 13: '2023-10-04', 14: '2023-10-18', 15: '2023-11-01'}
16
508


In [193]:
def clean_df(df):
    #remove all rows with no values
    df = df.dropna(how="all")
   # print(df)

    # get only expenses
    df2 = df.iloc[:,0:total_cols:2]
    #print(df2) 

    # get all expenses
    df2 = df2.melt().drop('variable',axis=1).rename({"value": "expense"}, axis=1)
    df2 = df2.dropna(how="all")
   # print(df2)


    df4 = df.iloc[:,1:total_cols:2]
    df5 = df4.melt().drop('variable',axis=1).rename({"value": "amount"}, axis=1)
    df5 = df5.dropna(how="all")
   # print(df4)
    #print(df5)

   # print(df2)
    df2["amount"] = df5.abs()
    df2.columns = ["expense", "amount"]

    df2 = df2.groupby("expense", as_index=False).sum()

#    i = df2[(df2["expense"] == "Income")]
#    df2 = df2.drop(i.index, axis=0)

 #   df2 = df2.sort_values(by=["amount"], ascending = False)

    return df2

df1 = df.iloc[0:cycle, :]
df1 = df1.drop(REMOVE, axis=0)
#print(df1)
#clean_df(df1)

In [194]:

def make_graph(df):
    plot = df.plot.pie(y="amount", labels=df["expense"], autopct='%1.0f%%')
   # plt.show()

In [195]:
#print(df.iloc[0:cycle, :])
#REMOVE = [11, 15, 27, 28]

REMOVE = [11, 15, 27]
#print(df.iloc[15, 0])

In [196]:
df_dict = {}
i = 0
writer = pd.ExcelWriter("budget14oct.xlsx", engine = 'openpyxl', mode='a', if_sheet_exists='overlay')
while (i < (no_cycles)):
    df1 = df.iloc[0:cycle, :]
    df1 = df1.drop(REMOVE, axis=0)
  #  print(df1)
    df1 = clean_df(df1)
    df_dict[i] = df1
    #print(df1)


    df2 = pd.DataFrame({'expense': [cycle_dates[i]], 'amount': [0]})
    df1 = categories.set_index('expense').add(df1.set_index('expense'), fill_value=0).reset_index()
    df3 = pd.concat([df2, df1])
  
    df3.to_excel(writer, sheet_name = 'test2', startrow = i//4*18, startcol = i%4*3+3, index=False)
  #  make_graph(df1)

    df = df.tail(-32)
    df.reset_index(drop=True, inplace=True)
    
    i = i + 1
writer.close()